### Evaluación Científico de Datos Jr 

Modelo predictivo que muestre la probabilidad de que un
CLIENTE retire el 70% o más de su saldo en los siguientes 3 meses

-Primer paso
Se realiza la lectura de los saldos y se busca estandarizar la información para un manejo más sencillo de los datos. 
Realizamos una agrupación a nivel ID del cliente para saber su saldo, qué contratos y que productos tiene activos, incluyendo el historial de su saldo mensual

In [27]:
import pyarrow.parquet as pq
import pandas as pd
from datetime import datetime as dt
import numpy

saldos = pq.read_table('0saldos.parquet').to_pandas()
saldos = saldos.loc[saldos['TipoDocum'] == 'C ']
# saldos
# Realizar la agrupación y especificar cómo agregar las columnas utilizando regex
# resultados_agrupados = df.groupby('columna_agrupadora').apply(lambda x: x.filter(regex=regex_a_mantener))

conteo_contratos_por_cliente = saldos.groupby('NroDocum').agg(
    Contrato=('Contrato', list),
    Productos=('PlanProducto', list),
    **{col: (col, 'first') for col in saldos.columns if col not in ['NroDocum', 'Contrato', 'PlanProducto', 'TipoDocum', 'SALDO_202209']}
)
# conteo_contratos_por_cliente['Saldo2022Aug'] = conteo_contratos_por_cliente['Saldo2022Aug'].apply(lambda x: list(set(x)))
conteo_contratos_por_cliente['Productos'] = conteo_contratos_por_cliente['Productos'].apply(lambda x: list(set(x)))

conteo_contratos_por_cliente.head(20)

,Contrato,Productos,SALDO_202101,SALDO_2021O2,SALDO_2021O3,SALDO_2021O4,SALDO_2021O5,SALDO_2021O6,SALDO_2021O7,SALDO_2021O8,...,SALDO_202112,SALDO_202201,SALDO_202202,SALDO_202203,SALDO_202204,SALDO_202205,SALDO_202206,SALDO_202207,SALDO_202208,SALDO_202210
NroDocum,,,,,,,,,,,,,,,,,,,,,
1000001634,[9950268],[4899],47092297.99,46315661.29,45229127.61,45298456.10,47140239.11,4.692357e+07,5.173611e+07,5.086159e+07,...,5.161528e+07,4.999541e+07,5.090699e+07,4.823857e+07,50249882.12,48430969.50,48976166.48,50399995.40,49148602.10,4.967498e+07
1000004345,"[8629082, 6261343, 6133677, 9374655, 5919728]","[3376, 8404]",0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000005362,[6845989],[7256],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000005396,[5835760],[7256],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000007479,[2779865],[8404],90932411.58,93993197.01,96553396.47,99759806.09,99628363.77,1.088142e+08,1.089287e+08,1.152094e+08,...,1.269493e+08,1.304447e+08,1.304298e+08,1.365314e+08,36996395.24,39371555.32,42176765.55,45424357.88,50964795.04,2.548386e+08
1000007557,[7095683],[7256],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000008577,[4620527],[6803],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000008826,"[2712448, 7337966, 5048101]",[7256],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00
1000009088,[5863862],[7256],0.00,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.000000e+00


Ahora leeremos las transferencias, dado que necesitamos el valor neto de las operaciones por mes, agruparemos la información a nivel mes para tener un historial ordenado.

In [8]:
transferencias = pq.read_table('0transferencias.parquet').to_pandas()
transferencias.drop(
    columns=['Usuario', 'CodDestino', 'CodTipoGiro', 'CodBancoDestinoACH', 'CodBancoSkandia', 'TipoCtaACH', 'TipoOper',
             'EstadoReverso'], inplace=True)
transferencias.sort_values(by='FechaEfectiva', ascending=True, inplace=True)
transferencias.FechaEfectiva = pd.to_datetime(transferencias.FechaEfectiva, format='%Y-%m-%d')

# Agrupar las transacciones por mes y sumar los valores
transferencias = transferencias.groupby(
    [transferencias['Contrato'], transferencias['FechaEfectiva'].dt.to_period('M')], as_index=False
).agg(
    FechaEfectiva=('FechaEfectiva', 'first'),
    ValorTransaccion=('ValorNeto', 'sum'),
    Productos=('PlanProducto', list)
)
transferencias['FechaEfectiva'] = transferencias['FechaEfectiva'].dt.strftime('%Y-%m-01')
transferencias['Productos'] = transferencias['Productos'].apply(lambda x: list(set(x)))
transferencias

,Contrato,FechaEfectiva,ValorTransaccion,Productos
0,10359,2021-01-01,10000000.00,[4899]
1,10359,2021-08-01,10000000.00,[4899]
2,10359,2022-06-01,50000000.00,[4899]
3,10359,2022-07-01,-45120000.00,[4899]
4,10359,2022-10-01,-25000000.00,[4899]
...,...,...,...,...
207128,9999852,2021-11-01,-33008997.44,[7256]
207129,9999884,2021-01-01,-33384841.00,[4899]
207130,9999898,2021-07-01,7000000.00,[7256]
207131,9999898,2021-09-01,5000000.00,[7256]


Al realzar la lectura del registro de clientes, vemos que no coincide con nuestro registro de saldos. Para mantener coherencia en la información eliminaremos las entradas sobrantes.
Realizaremos una fusion entre el dataframe de clientes y saldos. 

In [23]:

clientes = pq.read_table('0clientes.parquet').to_pandas()
clientes = clientes.loc[clientes['TIPODOCUM'] == 'C']
clientes.drop(
    columns=['EnvioExtractos'], inplace=True)

missings = list(set(saldos.NroDocum.drop_duplicates().values) - set(clientes.NroDocum.values))
print(missings)
mask = clientes['NroDocum'].isin(missings)
clientes = clientes[~mask]

clientes

[1070060609, 1199426185, 1105436267, 1067667956, 1191709847, 1168295676, 1114592927]


,TIPODOCUM,CIUDAD,FecNacim,NroDocum
0,C,"BOGOTA D.C., BOGOTA",1954-12-16,1092070504
1,C,"BOGOTA D.C., BOGOTA",1969-08-04,1097627287
2,C,"BOGOTA D.C., BOGOTA",1961-08-15,1066619616
3,C,"BOGOTA D.C., BOGOTA",1986-02-08,1079454835
4,C,"MEDELLIN, ANTIOQUIA",1981-01-15,1199126816
...,...,...,...,...
108087,C,"BOGOTA D.C., BOGOTA",1977-09-25,1077819263
108088,C,"BOGOTA D.C., BOGOTA",1964-09-30,1050991361
108089,C,"LA MESA, CUNDINAMARCA",1964-11-23,1117842187
108090,C,"BOGOTA D.C., BOGOTA",1976-07-01,1065109336
